### Compare Listing 
<li>a: vector uniform</li>
<li>b: greedy</li>
<li>c: e - greedy</li>
<li>d: Linear Reward Penalty</li>
<li>e: Linear Reward Inaction</li>
<li>f: UBC</li>
<li>g: BayesianUCB</li>
<li>h: Thompson Sampling</li>
<li>i: Neural Network</li>
<li>j: Non Stationary</li>

In [6]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import scipy,time
np.random.seed(5678)
np.set_printoptions(3)

In [17]:
# setting the ground truth
gt_prob = np.random.uniform(0,1,50)
optimal_choice = np.argmax(gt_prob)
num_ep  = 10
num_iter= 2000
print(gt_prob)
print('Best Choice: ',optimal_choice,gt_prob[optimal_choice])

[0.532 0.014 0.518 0.435 0.862 0.775 0.504 0.15  0.234 0.886 0.61  0.482
 0.711 0.115 0.16  0.47  0.101 0.817 0.625 0.841 0.915 0.729 0.353 0.514
 0.413 0.442 0.924 0.69  0.885 0.348 0.867 0.367 0.093 0.053 0.477 0.806
 0.524 0.819 0.865 0.683 0.104 0.447 0.674 0.627 0.997 0.326 0.523 0.592
 0.374 0.393]
Best Choice:  44 0.9972439200920686


In [19]:
# vectorized
a_reward = np.zeros((num_ep,num_iter))
a_choice = np.zeros((num_ep,num_iter))

for eps in range(num_ep):
    temp_reward = np.zeros(num_iter)
    temp_choice = np.zeros(num_iter)
    for iter in range(num_iter):
        

# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html